## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.68,74,14,10.54,light rain
1,1,Tiksi,RU,71.6872,128.8694,32.31,99,100,11.23,light snow
2,2,Qaanaaq,GL,77.4840,-69.3632,22.35,85,38,2.93,scattered clouds
3,3,Kosai,JP,34.7105,137.5316,73.56,60,84,9.35,broken clouds
4,4,Dikson,RU,73.5069,80.5464,28.35,74,100,6.91,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Cabo San Lucas,MX,22.8909,-109.9124,81.61,50,0,12.66,clear sky
16,16,Vaini,TO,-21.2000,-175.2000,80.76,89,75,5.75,broken clouds
22,22,Butaritari,KI,3.0707,172.7902,81.57,77,100,7.18,overcast clouds
24,24,Laguna,US,38.4210,-121.4238,99.28,13,0,13.80,clear sky
25,25,Hilo,US,19.7297,-155.0900,84.27,67,75,11.50,broken clouds
39,39,Minab,IR,27.1467,57.0801,83.61,58,0,3.33,clear sky
50,50,Praxedis Guerrero,MX,31.3667,-106.0167,87.26,5,0,25.93,clear sky
52,52,Georgetown,MY,5.4112,100.3354,85.95,74,20,2.30,thunderstorm
54,54,Kahului,US,20.8947,-156.4700,80.62,68,40,19.57,scattered clouds
55,55,Kavieng,PG,-2.5744,150.7967,84.67,66,100,10.65,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                139
City                   139
Country                139
Lat                    139
Lng                    139
Max Temp               139
Humidity               139
Cloudiness             139
Wind Speed             139
Current Description    139
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                139
City                   139
Country                139
Lat                    139
Lng                    139
Max Temp               139
Humidity               139
Cloudiness             139
Wind Speed             139
Current Description    139
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Cabo San Lucas,MX,81.61,clear sky,22.8909,-109.9124,
16,Vaini,TO,80.76,broken clouds,-21.2000,-175.2000,
22,Butaritari,KI,81.57,overcast clouds,3.0707,172.7902,
24,Laguna,US,99.28,clear sky,38.4210,-121.4238,
25,Hilo,US,84.27,broken clouds,19.7297,-155.0900,
39,Minab,IR,83.61,clear sky,27.1467,57.0801,
50,Praxedis Guerrero,MX,87.26,clear sky,31.3667,-106.0167,
52,Georgetown,MY,85.95,thunderstorm,5.4112,100.3354,
54,Kahului,US,80.62,scattered clouds,20.8947,-156.4700,
55,Kavieng,PG,84.67,overcast clouds,-2.5744,150.7967,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.count()

City                   139
Country                139
Max Temp               139
Current Description    139
Lat                    139
Lng                    139
Hotel Name             139
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
#https://www.adamsmith.haus/python/answers/how-to-drop-empty-rows-from-a-pandas-dataframe-in-python
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)


In [11]:
hotel_df.count()

City                   129
Country                129
Max Temp               129
Current Description    129
Lat                    129
Lng                    129
Hotel Name             129
dtype: int64

In [12]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Cabo San Lucas,MX,81.61,clear sky,22.8909,-109.9124,Vista Roof Restaurant
16,Vaini,TO,80.76,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
22,Butaritari,KI,81.57,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
24,Laguna,US,99.28,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
25,Hilo,US,84.27,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
689,San Ramon,US,96.26,clear sky,37.7799,-121.9780,San Ramon Marriott
693,Sattur,IN,81.16,broken clouds,9.3667,77.9333,Jaypriya Residency
697,Kapit,MY,84.18,broken clouds,2.0167,112.9333,Hotel Meligai Kapit
698,Lhokseumawe,ID,80.56,overcast clouds,5.1801,97.1507,OYO 3278 Penginapan Alfajri


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))